# **Arxiv metadata Analytics with PySpark RDD: JSON case study**

### Udemy Course: Best Hands-on Big Data Practices and Use Cases using PySpark

### Author: Amin Karami (PhD, FHEA)
#### email: amin.karami@ymail.com

In [1]:
########## ONLY in Colab ##########
!pip3 install pyspark
########## ONLY in Colab ##########

In [2]:
# Initializing Spark
from pyspark import SparkContext, SparkConf

conf1 = SparkConf().setAppName("Archive_Pyspark").setMaster("local[*]")
sc = SparkContext(conf=conf1)

print(sc)

print("ready to go")

23/03/19 15:05:50 WARN Utils: Your hostname, Adrian-Laptop.local resolves to a loopback address: 127.0.0.1; using 192.168.100.19 instead (on interface en0)
23/03/19 15:05:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 15:05:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
<SparkContext master=local[*] appName=Archive_Pyspark>
ready to go


In [3]:
# Read and Load Data to Spark
# Data source: https://www.kaggle.com/Cornell-University/arxiv/version/62
import json

rdd_json = sc.textFile("data.json", 100)
rdd = rdd_json.map(lambda x: json.loads(x))

rdd.persist()



PythonRDD[2] at RDD at PythonRDD.scala:53

In [4]:
# Check the number of parallelism and partitions:
print(sc.defaultParallelism)
print(rdd.getNumPartitions())

8
100


## Question 1: Count elements

In [6]:
rdd.count()

23/03/19 15:06:42 WARN MemoryStore: Not enough space to cache rdd_2_25 in memory! (computed 1608.9 KiB so far)
23/03/19 15:06:42 WARN BlockManager: Block rdd_2_25 could not be removed as it was not found on disk or in memory
23/03/19 15:06:42 WARN BlockManager: Putting block rdd_2_25 failed
23/03/19 15:06:43 WARN MemoryStore: Not enough space to cache rdd_2_21 in memory! (computed 1522.2 KiB so far)
23/03/19 15:06:43 WARN BlockManager: Block rdd_2_21 could not be removed as it was not found on disk or in memory
23/03/19 15:06:43 WARN BlockManager: Putting block rdd_2_21 failed
23/03/19 15:06:43 WARN MemoryStore: Not enough space to cache rdd_2_28 in memory! (computed 2.4 MiB so far)
23/03/19 15:06:43 WARN MemoryStore: Not enough space to cache rdd_2_23 in memory! (computed 2.4 MiB so far)


23/03/19 15:06:43 WARN BlockManager: Block rdd_2_28 could not be removed as it was not found on disk or in memory
23/03/19 15:06:43 WARN BlockManager: Putting block rdd_2_28 failed
23/03/19 15:06:43 WARN BlockManager: Block rdd_2_23 could not be removed as it was not found on disk or in memory
23/03/19 15:06:43 WARN BlockManager: Putting block rdd_2_23 failed
23/03/19 15:06:43 WARN MemoryStore: Not enough space to cache rdd_2_22 in memory! (computed 2.4 MiB so far)
23/03/19 15:06:43 WARN BlockManager: Block rdd_2_22 could not be removed as it was not found on disk or in memory
23/03/19 15:06:43 WARN BlockManager: Putting block rdd_2_22 failed
23/03/19 15:06:43 WARN MemoryStore: Not enough space to cache rdd_2_29 in memory! (computed 4.1 MiB so far)
23/03/19 15:06:43 WARN MemoryStore: Not enough space to cache rdd_2_30 in memory! (computed 4.2 MiB so far)
23/03/19 15:06:43 WARN BlockManager: Block rdd_2_29 could not be removed as it was not found on disk or in memory
23/03/19 15:06:43 W

23/03/19 15:06:43 WARN MemoryStore: Not enough space to cache rdd_2_31 in memory! (computed 2.4 MiB so far)
23/03/19 15:06:43 WARN BlockManager: Block rdd_2_31 could not be removed as it was not found on disk or in memory
23/03/19 15:06:43 WARN BlockManager: Putting block rdd_2_31 failed


23/03/19 15:06:45 WARN MemoryStore: Not enough space to cache rdd_2_32 in memory! (computed 1635.7 KiB so far)
23/03/19 15:06:45 WARN BlockManager: Block rdd_2_32 could not be removed as it was not found on disk or in memory
23/03/19 15:06:45 WARN BlockManager: Putting block rdd_2_32 failed
23/03/19 15:06:45 WARN MemoryStore: Not enough space to cache rdd_2_34 in memory! (computed 2.5 MiB so far)
23/03/19 15:06:45 WARN MemoryStore: Not enough space to cache rdd_2_35 in memory! (computed 1585.0 KiB so far)
23/03/19 15:06:45 WARN BlockManager: Block rdd_2_34 could not be removed as it was not found on disk or in memory
23/03/19 15:06:45 WARN BlockManager: Putting block rdd_2_34 failed
23/03/19 15:06:45 WARN BlockManager: Block rdd_2_35 could not be removed as it was not found on disk or in memory
23/03/19 15:06:45 WARN BlockManager: Putting block rdd_2_35 failed
23/03/19 15:06:45 WARN MemoryStore: Not enough space to cache rdd_2_36 in memory! (computed 2.5 MiB so far)


23/03/19 15:06:45 WARN BlockManager: Block rdd_2_36 could not be removed as it was not found on disk or in memory
23/03/19 15:06:45 WARN BlockManager: Putting block rdd_2_36 failed
23/03/19 15:06:45 WARN MemoryStore: Not enough space to cache rdd_2_38 in memory! (computed 2.4 MiB so far)
23/03/19 15:06:45 WARN BlockManager: Block rdd_2_38 could not be removed as it was not found on disk or in memory
23/03/19 15:06:45 WARN BlockManager: Putting block rdd_2_38 failed
23/03/19 15:06:45 WARN MemoryStore: Not enough space to cache rdd_2_39 in memory! (computed 4.3 MiB so far)
23/03/19 15:06:45 WARN BlockManager: Block rdd_2_39 could not be removed as it was not found on disk or in memory
23/03/19 15:06:45 WARN BlockManager: Putting block rdd_2_39 failed
23/03/19 15:06:45 WARN MemoryStore: Not enough space to cache rdd_2_37 in memory! (computed 4.4 MiB so far)
23/03/19 15:06:45 WARN BlockManager: Block rdd_2_37 could not be removed as it was not found on disk or in memory
23/03/19 15:06:45 W

...

23/03/19 15:07:01 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_2_96 in memory.
23/03/19 15:07:01 WARN MemoryStore: Not enough space to cache rdd_2_95 in memory! (computed 1702.4 KiB so far)
23/03/19 15:07:01 WARN BlockManager: Block rdd_2_95 could not be removed as it was not found on disk or in memory
23/03/19 15:07:01 WARN BlockManager: Putting block rdd_2_95 failed
23/03/19 15:07:01 WARN MemoryStore: Not enough space to cache rdd_2_96 in memory! (computed 0.0 B so far)
23/03/19 15:07:01 WARN BlockManager: Block rdd_2_96 could not be removed as it was not found on disk or in memory
23/03/19 15:07:01 WARN BlockManager: Putting block rdd_2_96 failed


23/03/19 15:07:02 WARN MemoryStore: Not enough space to cache rdd_2_89 in memory! (computed 9.2 MiB so far)
23/03/19 15:07:02 WARN BlockManager: Block rdd_2_89 could not be removed as it was not found on disk or in memory
23/03/19 15:07:02 WARN BlockManager: Putting block rdd_2_89 failed


23/03/19 15:07:04 WARN MemoryStore: Not enough space to cache rdd_2_98 in memory! (computed 3.2 MiB so far)
23/03/19 15:07:04 WARN BlockManager: Block rdd_2_98 could not be removed as it was not found on disk or in memory
23/03/19 15:07:04 WARN BlockManager: Putting block rdd_2_98 failed


23/03/19 15:07:04 WARN MemoryStore: Not enough space to cache rdd_2_97 in memory! (computed 5.2 MiB so far)
23/03/19 15:07:04 WARN BlockManager: Block rdd_2_97 could not be removed as it was not found on disk or in memory
23/03/19 15:07:04 WARN BlockManager: Putting block rdd_2_97 failed
23/03/19 15:07:04 WARN MemoryStore: Not enough space to cache rdd_2_99 in memory! (computed 5.9 MiB so far)
23/03/19 15:07:04 WARN BlockManager: Block rdd_2_99 could not be removed as it was not found on disk or in memory
23/03/19 15:07:04 WARN BlockManager: Putting block rdd_2_99 failed


2011231

## Question 2: Get the first two records


In [ ]:
rdd.take(2)

## Question 3: Get all attributes


In [ ]:
rdd.flatMap(lambda x: x.key()).distinct().collect()

## Question 4: Get the name of the licenses

## Question 5: Get the shortest and the longest titles

## Question 6: Find abbreviations with 5 or more letters in the abstract

## Question 7: Get the number of archive records per month ('update_date' attribute)

## Question 8: Get the average number of pages